# Loading entities files


### BERN entities

In [1]:
import pandas as pd
import ast
import os

In [2]:

# Loading classified entities

paths = (r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\ENTS\BERN_Sample10k\All_ENT_Types_V2\All_Bern_Type_Disease_V2.csv",
r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\ENTS\BERN_Sample10k\All_ENT_Types_V2\All_Bern_Type_Drug_V2.csv",
r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\ENTS\BERN_Sample10k\All_ENT_Types_V2\All_Bern_Type_Gene_V2.csv",
r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\ENTS\BERN_Sample10k\All_ENT_Types_V2\All_Bern_Type_Species_V2.csv")

def load_file(path, type_):
    with open (path, 'r', encoding = 'utf-8') as file:
        data = pd.read_csv(file)
    return pd.DataFrame(data[type_].unique())  # get the unique

disease_bern = load_file(paths[0], 'disease' )
drug_bern = load_file(paths[1], 'drug' )
gene_bern = load_file(paths[2], 'gene' )
species_bern = load_file(paths[3], 'species' )
print(len(disease_bern), len(drug_bern), len(gene_bern), len(species_bern))

8129 4753 16888 1809


### UMLS entities

In [3]:
# Loading metamap entities 
import pandas as pd
paths = [r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\ENTS\MetaMap\Output\cleanedNrefined-entities\meta_refined_entities_pdf.csv",
r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\ENTS\MetaMap\Output\cleanedNrefined-entities\meta_refined_entities_pmc.csv"]

with open(paths[0], 'r', encoding = 'utf-8') as f:
    pdf_ents = pd.read_csv(f)
    
with open(paths[1], 'r', encoding = 'utf-8') as f:
    pmc_ents = pd.read_csv(f)
    


In [4]:
pmc_ents.head()

,Unnamed: 0,matchedtext,CUI,Preferred Name,synonym,Semantic types
0,0,coronavirus,C0206419,Genus: Coronavirus,NaN,virs
1,1,severe acute respiratory syndrome coronavirus,C1175743,SARS coronavirus,NaN,virs
2,2,coronavirus,C0206419,Genus: Coronavirus,NaN,virs
3,3,disease,C0012634,Disease,NaN,dsyn
4,4,april,C3715025,TNFSF13 wt Allele,NaN,gngm


In [5]:
aapp,anim,celc,cell,clnd,dsyn,gngm,sosy,topp,virs = [x for _, x in pmc_ents.groupby(pmc_ents['Semantic types'])]
aapp1,anim1,celc1,cell1,clnd1,dsyn1,gngm1,sosy1,topp1,virs1 = [x for _, x in pdf_ents.groupby(pdf_ents['Semantic types'])]


In [6]:
disease_meta = pd.concat([dsyn,virs,dsyn1,virs1])
gene_prot_meta = pd.concat([aapp,cell,celc,gngm,aapp1,cell1,celc1,gngm1])
ani_species_meta = pd.concat([anim,anim1])
symptom_meta = pd.concat([sosy,sosy1])
treat_proc_meta = pd.concat([topp,topp1])
drug_meta = pd.concat([clnd,clnd1])

In [7]:
print(len(disease_meta), len(gene_prot_meta), len(ani_species_meta), len(symptom_meta), len(treat_proc_meta), len(drug_meta))

6711358 13736269 216123 635553 5223685 27482


#### Combine bern and metamap 

In [8]:
disease = pd.concat([disease_bern,disease_meta['matchedtext']])[0].to_list()
gene_prot = pd.concat([gene_bern,gene_prot_meta ['matchedtext']])[0].to_list()
ani_species = pd.concat([species_bern, ani_species_meta ['matchedtext']])[0].to_list()
symptom = symptom_meta ['matchedtext'].to_list()
treat_proc = treat_proc_meta ['matchedtext'].to_list()
drug = pd.concat([drug_bern,drug_meta ['matchedtext']])[0].to_list()

In [9]:
# Entities with duplicate
print(len(disease), len(gene_prot), len(ani_species), len(symptom), len(treat_proc), len(drug))

6719487 13753157 217932 635553 5223685 32235


In [10]:
# Entities without duplicate
#print(len(disease[0].unique()), len(gene_prot[0].unique()), len(ani_species[0].unique()), len(symptom.unique()), len(treat_proc.unique()), len(drug[0].unique()))
print(len(set(disease)), len(set(gene_prot)), len(set(ani_species)), len(set(symptom)), len(set(treat_proc)), len(set(drug)))

25700 41928 1900 1678 5708 5351


### Turn into a set


In [11]:
disease = list(set(disease))
gene_prot = list(set(gene_prot))
ani_species = list(set(ani_species))
symptom = list(set(symptom))
treat_proc = list(set(treat_proc))
drug = list(set(drug))

#### Removing entities with names: disease, drug, gene, symptom

In [12]:
def removeItem (list_):
    removing_list = ['disease', 'drug', 'gene', 'symptom']
    for i in removing_list:
        if i in list_:
            list_.remove(i)
    return list_

symptom = removeItem(symptom)
disease = removeItem(disease)
gene_prot =  removeItem(gene_prot)
ani_species = removeItem(ani_species)
treat_proc = removeItem(treat_proc)
drug = removeItem(drug)
print(len(disease), len(gene_prot), len(ani_species), len(symptom), len(treat_proc), len(drug))

25699 41927 1900 1677 5707 5351


# Loading OpenIE triples



dirs=[r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\OpenIE\IEtriples_pdf\output",
r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\OpenIE\IEtriples_pmc\OpenIE_triples"]

count_all_triples = 0
frames_pdf = []
frames_pmc = []
filenames = os.listdir(dirs[1])
print('number of files: ',len(filenames))
print("Extracted from the Number of sentences:", (len(filenames) *300))
for filename in filenames:
    filename = os.path.join(dirs[1], filename)
    filename.encode('unicode-escape')
    try:
        data_triples = pd.read_csv(open(filename, 'r', encoding='utf-8'))
        count_all_triples += len(data_triples)
        frames_pdf.append(data_triples)
    except:
        print(filename)
pdf_trip_df = pd.concat(frames_pdf)
            
#print(all_refined_ieTrips)
print('the number of triples: ', len( pdf_trip_df))
print('triples in total: ', count_all_triples)     
pdf_trip_df.head()

#with open(dirs[0]+'//'+'OIE_pdf.csv', 'w', newline = '', encoding = 'utf-8') as f:
    pdf_trip_df.to_csv(f)
    
with open(dirs[1]+'//'+'OIE_pmc.csv', 'w', newline = '', encoding = 'utf-8') as f:
    pdf_trip_df.to_csv(f)

In [13]:
ie_path = [r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\OpenIE\processed_IEtriples\OIE_pdf.csv",
r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\OpenIE\processed_IEtriples\OIE_pmc.csv"]
with open (ie_path[0],'r', encoding = 'utf-8' ) as file:
    pdf_trip_df = pd.read_csv(file)
            
print('the number of triples: ', len( pdf_trip_df))

with open (ie_path[1],'r', encoding = 'utf-8' ) as file:
    pmc_trip_df = pd.read_csv(file)
print('the number of triples: ', len( pmc_trip_df))
pmc_trip_df.head()


the number of triples:  30888691
the number of triples:  7773855


,Unnamed: 0,Unnamed: 0.1,subject,relation,object
0,0,0,may precursor event,are,frequent findings
1,1,1,may precursor event,initiating,suppurate AOM
2,2,2,may precursor event,are,findings
3,3,3,OME,initiating,suppurate AOM
4,4,4,may precursor event,are,findings in OME 54


### Mapping with pdf

In [15]:
copy_pdf_trip_df = pdf_trip_df.copy()
# Pseudo change entities into entities types
## In subjects
pdf_trip_df['subject'].mask(pdf_trip_df['subject'].isin(drug),'drug', inplace = True)
pdf_trip_df['subject'].mask(pdf_trip_df['subject'].isin(symptom),'symptom', inplace = True)
pdf_trip_df['subject'].mask(pdf_trip_df['subject'].isin(ani_species),'species', inplace = True)  
pdf_trip_df['subject'].mask(pdf_trip_df['subject'].isin(treat_proc),'treat_procedure', inplace = True)
pdf_trip_df['subject'].mask(pdf_trip_df['subject'].isin(disease),'disease', inplace = True)
pdf_trip_df['subject'].mask(pdf_trip_df['subject'].isin(gene_prot),'gene', inplace = True)
                                                  
pdf_trip_df['subject'].value_counts()


disease                                                 1316745
we                                                       888150
species                                                  790278
gene                                                     771322
it                                                       626260
                                                         ...   
reacting 8a                                                   1
e.g.nephrology                                                1
severe pulmonary shunt fraction                               1
CO2 emissions data                                            1
significant downregulation proinflammatory cytokines          1
Name: subject, Length: 1969563, dtype: int64

In [16]:
pdf_trip_df['object'].mask(pdf_trip_df['object'].isin(drug),'drug', inplace = True)
pdf_trip_df['object'].mask(pdf_trip_df['object'].isin(symptom),'symptom', inplace = True)
pdf_trip_df['object'].mask(pdf_trip_df['object'].isin(ani_species),'species', inplace = True)  
pdf_trip_df['object'].mask(pdf_trip_df['object'].isin(treat_proc),'treat_procedure', inplace = True)
pdf_trip_df['object'].mask(pdf_trip_df['object'].isin(disease),'disease', inplace = True)
pdf_trip_df['object'].mask(pdf_trip_df['object'].isin(gene_prot),'gene', inplace = True)
pdf_trip_df['object'].value_counts()

disease                                             486891
gene                                                270791
species                                             232327
treat_procedure                                     174698
drug                                                103627
                                                     ...  
presence of target applied                               1
when produced in presence                                1
safe transport of potentially high risk patients         1
apical routes.Sorting to pathway                         1
well established international partners                  1
Name: object, Length: 11795318, dtype: int64

# Change dynamic relations into predefined relations

#ie_triples['object'].mask(ie_triples['object'].isin(disease[0]),'disease', inplace = True)

pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='disease') & (pdf_trip_df['object']=='gene'),'have_gene', inplace = True)
pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='gene') & (pdf_trip_df['object']=='desease'), 'be_in', inplace = True)
pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='disease') & (pdf_trip_df['object']=='drug'), 'be_treated_by', inplace = True)
pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='drug') & (pdf_trip_df['object']=='disease'), 'treat', inplace = True)
pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='disease') & (pdf_trip_df['object']=='species'), 'be_transmited_by', inplace = True)
pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='species') & (pdf_trip_df['object']=='disease'), 'transmit', inplace = True)
pdf_trip_df['relation'].mask((pdf_trip_df['subject']=='disease') & (pdf_trip_df['object']=='disease'), 'be_related_with', inplace = True)

In [18]:


types = ['disease', 'gene', 'drug', 'symptom', 'treat_procedure', 'species']

i=0
j=i+1
for i in range(len(types)):
    for j in range(len(types)):
        print(types[i], types[j])
        pdf_trip_df['relation'].mask((pdf_trip_df['subject']== types[i]) & (pdf_trip_df['object']== types[j]), types[i] +'_'+ types[j], inplace = True)
        #new_try['relation'].mask((new_try['subject']== types[i]) & (new_try['object']== types[j]), types[i] +'_'+ types[j], inplace = True)

   

disease disease
disease gene
disease drug
disease symptom
disease treat_procedure
disease species
gene disease
gene gene
gene drug
gene symptom
gene treat_procedure
gene species
drug disease
drug gene
drug drug
drug symptom
drug treat_procedure
drug species
symptom disease
symptom gene
symptom drug
symptom symptom
symptom treat_procedure
symptom species
treat_procedure disease
treat_procedure gene
treat_procedure drug
treat_procedure symptom
treat_procedure treat_procedure
treat_procedure species
species disease
species gene
species drug
species symptom
species treat_procedure
species species


In [19]:
pdf_trip_df['relation'].value_counts()


is                                1812026
was                               1246242
were                              1188493
are                               1042017
is in                              900814
                                   ...   
fourth family of                        1
Very rarely may cause                   1
certainly lies within                   1
been conserved across                   1
were tested retrospectively by          1
Name: relation, Length: 1473299, dtype: int64

In [20]:
new_relation = [i for i in pdf_trip_df['relation']]
copy_pdf_trip_df['new_relation'] = new_relation
copy_pdf_trip_df

,Unnamed: 0,Unnamed: 0.1,subject,relation,object,new_relation
0,0,0,Direct contact,contaminated with,subsequent contact with cavity,contaminated with
1,1,1,Direct contact,is with,person,is with
2,2,2,contact,contaminated with,contact,contaminated with
3,3,3,subsequent contact,is with,own nasal cavity,is with
4,4,4,Direct contact,contaminated with,subsequent contact,contaminated with
...,...,...,...,...,...,...
30888686,1606,1606,acquisition,might,for pathogens might required.Simple blood,might
30888687,1607,1607,acquisition,complicate field operations in,midst of epidemic,complicate field operations in
30888688,1608,1608,live pathogens,acquisition of be,for pathogens might required.Simple blood,acquisition of be
30888689,1609,1609,acquisition,logistical issues regarding,processing of acute infectious samples,logistical issues regarding


In [21]:
copy_pdf_trip_df['new_relation'].value_counts()
copy_pdf_trip_df.drop(columns = ['relation', 'Unnamed: 0'], inplace = True)
copy_pdf_trip_df

,Unnamed: 0.1,subject,object,new_relation
0,0,Direct contact,subsequent contact with cavity,contaminated with
1,1,Direct contact,person,is with
2,2,contact,contact,contaminated with
3,3,subsequent contact,own nasal cavity,is with
4,4,Direct contact,subsequent contact,contaminated with
...,...,...,...,...
30888686,1606,acquisition,for pathogens might required.Simple blood,might
30888687,1607,acquisition,midst of epidemic,complicate field operations in
30888688,1608,live pathogens,for pathogens might required.Simple blood,acquisition of be
30888689,1609,acquisition,processing of acute infectious samples,logistical issues regarding


In [22]:
predef_relations = ['disease_disease','disease_gene','disease_drug','disease_symptom','disease_treat_procedure','disease_species','gene_disease','gene_gene','gene_drug','gene_symptom','gene_treat_procedure','gene_species',
'drug_disease','drug_gene','drug_drug','drug_symptom','drug_treat_procedure','drug_species','symptom_disease','symptom_gene','symptom_drug','symptom_symptom','symptom_treat_procedure','symptom_species',
'treat_procedure_disease','treat_procedure_gene','treat_procedure_drug','treat_procedure_symptom','treat_procedure_treat_procedure','treat_procedure_species']
new_triples = copy_pdf_trip_df.loc[(copy_pdf_trip_df['new_relation'].isin(predef_relations) )]


print(new_triples.new_relation.value_counts())

disease_disease                    44441
disease_species                    17783
gene_gene                          17616
gene_disease                       10113
disease_gene                        9311
disease_treat_procedure             7700
treat_procedure_disease             6312
drug_disease                        5801
gene_treat_procedure                4736
treat_procedure_treat_procedure     4678
gene_species                        4456
drug_gene                           4217
drug_drug                           3924
disease_symptom                     3881
treat_procedure_gene                3606
treat_procedure_species             3597
disease_drug                        3308
drug_species                        3258
symptom_symptom                     3253
symptom_disease                     3100
gene_drug                           2901
drug_treat_procedure                2715
treat_procedure_drug                2139
symptom_species                      989
gene_symptom    

In [23]:
print(len(new_triples))

175988


#### # Cleaning the triples

In [24]:
# Defining pre-processing function
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
def preprocessEntity (text):
    processed1 = lowercase (text)
    processed2 = lemmatization(processed1)
    normalized = normalizeCOVID (processed2)
    return normalized

def lowercase (text):
    lowercased = text.lower()
    return lowercased

def lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized = ' '.join([lemmatizer.lemmatize(w) for w in text.split()])
    return lemmatized

def normalizeCOVID (text):
    covid_variants1 = ['coronarivus disease', 'covid19 coronavirus',  'coronarivus', 'sars-cov-2', '2019-ncov']
    covid_variants2 = ['corona','sars-cov']
    for variant in covid_variants1:
        if variant in text:
            text = text.replace(variant, 'covid-19')
            
    if (('corona' in text) and ('coronavirus' not in text)):
            text = text.replace('corona', 'covid-19')
            
    if (('sars-cov' in text) and ('sars-cov-2' not in text)):
            text = text.replace('corona', 'covid-19')
    return text
        
    
preprocessEntity ('normalized bananas coronarivus disease coronarivus disease sars-cov-2')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\huyen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'normalized banana covid-19 covid-19 covid-19'

In [25]:
# drop null values
new_triples = new_triples.dropna(subset=['subject'])
new_triples = new_triples.dropna(subset=['object'])


# applying cleaning_f

new_triples['subject'] = new_triples[['subject']].applymap(preprocessEntity)
new_triples['object'] = new_triples[['object']].applymap(preprocessEntity)


print(len(new_triples.subject.unique()), len(new_triples.object.unique()))
print(len(new_triples.drop_duplicates(subset = None)))

9027 9624
175116


In [26]:
new_triples.new_relation.value_counts()

disease_disease                    44441
disease_species                    17783
gene_gene                          17616
gene_disease                       10113
disease_gene                        9311
disease_treat_procedure             7700
treat_procedure_disease             6312
drug_disease                        5794
gene_treat_procedure                4736
treat_procedure_treat_procedure     4678
gene_species                        4456
drug_gene                           4194
drug_drug                           3902
disease_symptom                     3881
treat_procedure_gene                3606
treat_procedure_species             3597
disease_drug                        3282
drug_species                        3254
symptom_symptom                     3253
symptom_disease                     3100
gene_drug                           2888
drug_treat_procedure                2705
treat_procedure_drug                2137
symptom_species                      989
gene_symptom    

In [27]:
with open(r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\triples_with_predefined_relations\new_triples_with_predefined_relations_pdf_June20_.csv", 'w', newline = '', encoding = 'utf-8') as f:
    new_triples.to_csv(f)

### pmc

In [29]:

copy_pmc_trip_df = pmc_trip_df.copy()
# Pseudo change entities into entities types
## In subjects
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(disease),'disease', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(drug),'drug', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(gene_prot),'gene', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(ani_species),'species', inplace = True)  
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(symptom),'symptom', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(treat_proc),'treat_procedure', inplace = True)
                                                  

pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(disease),'disease', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(drug),'drug', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(gene_prot),'gene', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(ani_species),'species', inplace = True)  
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(symptom),'symptom', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(treat_proc),'treat_procedure', inplace = True)

# Change dynamic relations into predefined relations

types = ['disease', 'gene', 'drug', 'symptom', 'treat_procedure', 'species']


i=0
j=i+1
for i in range(len(types)):
    for j in range(len(types)):
        print(types[i], types[j])
        pmc_trip_df['relation'].mask((pmc_trip_df['subject']== types[i]) & (pmc_trip_df['object']== types[j]), types[i] +'_'+ types[j], inplace = True)

                              

new_relation = [i for i in pmc_trip_df['relation']]
copy_pmc_trip_df['new_relation'] = new_relation


copy_pmc_trip_df['new_relation'].value_counts()
copy_pmc_trip_df.drop(columns = ['relation', 'Unnamed: 0'], inplace = True)



print(copy_pmc_trip_df['new_relation'].value_counts())

predef_relations = ['disease_disease','disease_gene','disease_drug','disease_symptom','disease_treat_procedure','disease_species','gene_disease','gene_gene','gene_drug','gene_symptom','gene_treat_procedure','gene_species',
'drug_disease','drug_gene','drug_drug','drug_symptom','drug_treat_procedure','drug_species','symptom_disease','symptom_gene','symptom_drug','symptom_symptom','symptom_treat_procedure','symptom_species',
'treat_procedure_disease','treat_procedure_gene','treat_procedure_drug','treat_procedure_symptom','treat_procedure_treat_procedure','treat_procedure_species']
new_triples2 = copy_pmc_trip_df.loc[(copy_pmc_trip_df['new_relation'].isin(predef_relations) )]


print(new_triples2.new_relation.value_counts())
print(len(new_triples2))


disease disease
disease gene
disease drug
disease symptom
disease treat_procedure
disease species
gene disease
gene gene
gene drug
gene symptom
gene treat_procedure
gene species
drug disease
drug gene
drug drug
drug symptom
drug treat_procedure
drug species
symptom disease
symptom gene
symptom drug
symptom symptom
symptom treat_procedure
symptom species
treat_procedure disease
treat_procedure gene
treat_procedure drug
treat_procedure symptom
treat_procedure treat_procedure
treat_procedure species
species disease
species gene
species drug
species symptom
species treat_procedure
species species
is                       414609
was                      339200
were                     331797
are                      258766
is in                    212573
                          ...  
fit data sets based           1
Duyn is at_time               1
Matthijnssens at_time         1
were as high among            1
believe?The here              1
Name: new_relation, Length: 466874, dtype: int64

In [30]:
new_triples2 = new_triples2.dropna(subset=['subject'])
new_triples2 = new_triples2.dropna(subset=['object'])
new_triples2['subject'] = new_triples2[['subject']].applymap(preprocessEntity)
new_triples2['object'] = new_triples2[['object']].applymap(preprocessEntity)
print(len(new_triples2.drop_duplicates(subset = None)))

54385


In [31]:
new_triples2

,Unnamed: 0.1,subject,object,new_relation
37,37,dysfunction,child,disease_species
39,39,et dysfunction,child,disease_species
67,67,et dysfunction,child,disease_species
190,190,rsv,virus,disease_disease
386,386,treatment,child,treat_procedure_species
...,...,...,...,...
7773163,809,virus,infection,disease_disease
7773684,1330,expression,eif2α,treat_procedure_gene
7773783,1429,expression,sg,treat_procedure_gene
7773846,1492,influenza a virus,iav,disease_disease


In [32]:

with open(r"C:\Users\huyen\OneDrive - UNT System\COURSES\INFO 5731\Final_Project\Group6_Working\AUG4\KG\triples_with_predefined_relations\new_triples_with_predefined_relations_pmc_June20_.csv", 'w', newline = '', encoding = 'utf-8') as f:
    new_triples2.to_csv(f)

In [33]:
#print('the number of refined pdf triples without duplicates: ', len(new_triples))
print('the number of refined pmc triples without duplicates: ', len(new_triples2))


the number of refined pmc triples without duplicates:  54451



predef_relations = [
copy_pmc_trip_df = pmc_trip_df.copy()
# Pseudo change entities into entities types
## In subjects
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(disease[0]),'disease', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(drug[0]),'drug', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(gene_prot[0]),'gene', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(ani_species[0]),'species', inplace = True)  
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(symptom),'symptom', inplace = True)
pmc_trip_df['subject'].mask(pmc_trip_df['subject'].isin(treat_proc),'treat_procedure', inplace = True)
                                                  

pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(disease[0]),'disease', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(drug[0]),'drug', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(gene_prot[0]),'gene', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(ani_species[0]),'species', inplace = True)  
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(symptom),'symptom', inplace = True)
pmc_trip_df['object'].mask(pmc_trip_df['object'].isin(treat_proc),'treat_procedure', inplace = True)

# Change dynamic relations into predefined relations

types = ['disease', 'gene', 'drug', 'symptom', 'treat_procedure', 'species']


i=0
j=i+1
for i in range(len(types)-1):
    for j in range(len(types)):
        print(types[i], types[j])
        pmc_trip_df['relation'].mask((pmc_trip_df['subject']== types[i]) & (pmc_trip_df['object']== types[j]), types[i] +'_'+ types[j], inplace = True)

                              

new_relation = [i for i in pmc_trip_df['relation']]
copy_pmc_trip_df['new_relation'] = new_relation


copy_pmc_trip_df['new_relation'].value_counts()
copy_pmc_trip_df.drop(columns = ['relation', 'Unnamed: 0'], inplace = True)



print(copy_pmc_trip_df['new_relation'].value_counts())

predef_relations = ['disease_disease','disease_gene','disease_drug','disease_symptom','disease_treat_procedure','disease_species','gene_disease','gene_gene','gene_drug','gene_symptom','gene_treat_procedure','gene_species',
'drug_disease','drug_gene','drug_drug','drug_symptom','drug_treat_procedure','drug_species','symptom_disease','symptom_gene','symptom_drug','symptom_symptom','symptom_treat_procedure','symptom_species',
'treat_procedure_disease','treat_procedure_gene','treat_procedure_drug','treat_procedure_symptom','treat_procedure_treat_procedure','treat_procedure_species']
new_triples2 = copy_pmc_trip_df.loc[(copy_pmc_trip_df['new_relation'].isin(predef_relations) )]


print(new_triples2.new_relation.value_counts())

